In [1]:
import json
import fitbitAPI
import pandas as pd
from datetime import datetime

## Fitbit API connection

In [ ]:
fitbit_api = fitbitAPI.connect()

## Data collection

Create the collection date range

In [ ]:
startTime = pd.datetime(year = 2023, month = 2, day = 15)
endTime = pd.datetime(year = 2023, month = 4, day = 15)
dates = pd.date_range(start=startTime, end = endTime)

#### Sleep data

In [4]:
sleep = []
for date in dates:
    date = date.date().strftime("%Y-%m-%d")

    sleep_data = fitbit_api.sleep(date=date)  # api call

    if sleep_data['sleep']:
        sleep_dict = {'type': 'sleep'}

        data = {'date': date}
        keys = ['duration', 'efficiency'] # select data from api
        data.update({x:sleep_data['sleep'][0][x] for x in keys}) # create a dict that is ONE Fitbit record/document in the collection

        keys = ['deep', 'light', 'rem', 'wake'] # select data from api
        data.update({x:sleep_data['summary']['stages'][x] for x in keys}) # create a dict that is ONE Fitbit record/document in the collection

        sleep_dict.update({'data': data})
        sleep.append(sleep_dict)
sleep

[{'type': 'sleep',
  'data': {'date': '2023-02-19',
   'duration': 16800000,
   'efficiency': 94,
   'deep': 44,
   'light': 145,
   'rem': 54,
   'wake': 37}},
 {'type': 'sleep',
  'data': {'date': '2023-02-21',
   'duration': 29220000,
   'efficiency': 94,
   'deep': 78,
   'light': 291,
   'rem': 59,
   'wake': 59}},
 {'type': 'sleep',
  'data': {'date': '2023-02-22',
   'duration': 34680000,
   'efficiency': 94,
   'deep': 87,
   'light': 306,
   'rem': 110,
   'wake': 75}},
 {'type': 'sleep',
  'data': {'date': '2023-02-23',
   'duration': 24780000,
   'efficiency': 97,
   'deep': 57,
   'light': 257,
   'rem': 48,
   'wake': 51}},
 {'type': 'sleep',
  'data': {'date': '2023-02-24',
   'duration': 29160000,
   'efficiency': 94,
   'deep': 105,
   'light': 230,
   'rem': 95,
   'wake': 56}},
 {'type': 'sleep',
  'data': {'date': '2023-02-26',
   'duration': 34020000,
   'efficiency': 95,
   'deep': 78,
   'light': 345,
   'rem': 79,
   'wake': 65}},
 {'type': 'sleep',
  'data': {'d

In [5]:
with open("data/sleep.json", "w") as outfile:
    json.dump(sleep, outfile)

#### Activity data

In [5]:
activity = []
for date in dates:
    date = date.date().strftime("%Y-%m-%d")
    activity_data = fitbit_api.activities(date=date)  # api call

    # general activity information
    activity_dict = {'type': 'activity'}
    data = {'date': date}
    keys = ['caloriesOut', 'floors', 'steps'] # select data from api
    data.update({x:activity_data['summary'][x] for x in keys})

    # minutes information
    minutes_dict = {}
    keys = ['fairlyActiveMinutes', 'sedentaryMinutes', 'veryActiveMinutes', 'lightlyActiveMinutes'] # select data from api
    minutes_dict.update({x:activity_data['summary'][x] for x in keys})
    minutes_dict["moderatelyActiveMinutes"] = minutes_dict.pop("fairlyActiveMinutes")
    data.update(minutes_dict)

    # distances information
    distances = activity_data['summary']['distances']
    distances_dict = {}
    for distance in distances:
        if distance['activity'] == 'veryActive':
            distances_dict.update({'veryActive_distance': distance['distance']})
        if distance['activity'] == 'moderatelyActive':
            distances_dict.update({'moderatelyActive_distance': distance['distance']})
        if distance['activity'] == 'lightlyActive':
            distances_dict.update({'lightlyActive_distance': distance['distance']})
        if distance['activity'] == 'sedentaryActive':
            distances_dict.update({'sedentaryActive_distance': distance['distance']})
    data.update(distances_dict) # create a dict that is ONE Fitbit record/document in the collection

    activity_dict.update({'data': data})
    activity.append(activity_dict)
activity

[{'type': 'activity',
  'data': {'date': '2023-02-15',
   'caloriesOut': 1851,
   'floors': 14,
   'steps': 5791,
   'sedentaryMinutes': 1230,
   'veryActiveMinutes': 8,
   'lightlyActiveMinutes': 200,
   'moderatelyActiveMinutes': 2,
   'veryActive_distance': 0.52,
   'moderatelyActive_distance': 0.03,
   'lightlyActive_distance': 2.04,
   'sedentaryActive_distance': 0}},
 {'type': 'activity',
  'data': {'date': '2023-02-16',
   'caloriesOut': 2297,
   'floors': 20,
   'steps': 14945,
   'sedentaryMinutes': 1107,
   'veryActiveMinutes': 27,
   'lightlyActiveMinutes': 268,
   'moderatelyActiveMinutes': 38,
   'veryActive_distance': 1.4,
   'moderatelyActive_distance': 1.68,
   'lightlyActive_distance': 3.46,
   'sedentaryActive_distance': 0}},
 {'type': 'activity',
  'data': {'date': '2023-02-17',
   'caloriesOut': 2013,
   'floors': 10,
   'steps': 8214,
   'sedentaryMinutes': 1171,
   'veryActiveMinutes': 11,
   'lightlyActiveMinutes': 257,
   'moderatelyActiveMinutes': 1,
   'veryAc

In [6]:
with open("data/activity.json", "w") as outfile:
    json.dump(activity, outfile)

#### Goals data

In [7]:
# create only one document without date information because I didn't change the goals
goals_dict = {'type': 'goals'}
keys = ['caloriesOut', 'steps', 'floors'] # select data from api
data = {x:activity_data['goals'][x] for x in keys}
goals_dict.update({'data': data})
goals_dict

{'type': 'goals', 'data': {'caloriesOut': 2200, 'steps': 8000, 'floors': 10}}

In [8]:
with open("data/goals.json", "w") as outfile:
    json.dump(goals_dict, outfile)

#### Badges data

In [4]:
badges_data = fitbit_api.get_badges() # there are duplicates but is normal - it means that user get 2 same badges in a day

In [5]:
badges = []
keys =['dateTime', 'badgeType'] # select data from api
for badge in badges_data['badges']:
    badge_dict = {'type': 'badges'}
    data = {x:badge[x] for x in keys}

    # keep the badges between 15-02-2023 and 15-04-2023
    date = data['dateTime']
    date = datetime.strptime(date, '%Y-%m-%d').date()
    if (date <= datetime.strptime('2023-04-15', '%Y-%m-%d').date()) and (date >= datetime.strptime('2023-02-15', '%Y-%m-%d').date()):
        data["date"] = data.pop("dateTime")
        badge_dict.update({'data': data})
        badges.append(badge_dict)
badges

[{'type': 'badges',
  'data': {'badgeType': 'DAILY_STEPS', 'date': '2023-02-26'}},
 {'type': 'badges',
  'data': {'badgeType': 'DAILY_STEPS', 'date': '2023-02-27'}},
 {'type': 'badges',
  'data': {'badgeType': 'DAILY_STEPS', 'date': '2023-03-24'}},
 {'type': 'badges',
  'data': {'badgeType': 'DAILY_STEPS', 'date': '2023-03-24'}},
 {'type': 'badges',
  'data': {'badgeType': 'DAILY_STEPS', 'date': '2023-04-11'}},
 {'type': 'badges',
  'data': {'badgeType': 'DAILY_FLOORS', 'date': '2023-03-24'}}]

In [6]:
with open("data/badges.json", "w") as outfile:
    json.dump(badges, outfile)

#### Heart data

In [9]:
heart = []
for date in dates:
    date = date.date().strftime("%Y-%m-%d")

    heart_data = fitbit_api.intraday_time_series('activities/heart', date, detail_level='1min') # api call
    heart_data = heart_data['activities-heart-intraday']['dataset'] # select data from api

    if heart_data:
        # aggregation of the daily information about heart rate
        heart_dict = {'type': 'heart'}
        df = pd.DataFrame(heart_data)
        min_value = df['value'].min()
        max_value = df['value'].max()
        mean_value = df['value'].mean()

        data = {'date': date, 'min_heart_rate': min_value, 'max_heart_rate': max_value, 'mean_heart_rate': mean_value}
        heart_dict.update({'data': data})
        heart.append(heart_dict)
heart

[{'type': 'heart',
  'data': {'date': '2023-02-15',
   'min_heart_rate': 64,
   'max_heart_rate': 159,
   'mean_heart_rate': 85.28256880733944}},
 {'type': 'heart',
  'data': {'date': '2023-02-16',
   'min_heart_rate': 49,
   'max_heart_rate': 161,
   'mean_heart_rate': 81.90396659707724}},
 {'type': 'heart',
  'data': {'date': '2023-02-17',
   'min_heart_rate': 59,
   'max_heart_rate': 150,
   'mean_heart_rate': 82.17624810892588}},
 {'type': 'heart',
  'data': {'date': '2023-02-18',
   'min_heart_rate': 53,
   'max_heart_rate': 132,
   'mean_heart_rate': 78.31213017751479}},
 {'type': 'heart',
  'data': {'date': '2023-02-19',
   'min_heart_rate': 55,
   'max_heart_rate': 119,
   'mean_heart_rate': 72.12554112554112}},
 {'type': 'heart',
  'data': {'date': '2023-02-20',
   'min_heart_rate': 58,
   'max_heart_rate': 130,
   'mean_heart_rate': 80.18756585879873}},
 {'type': 'heart',
  'data': {'date': '2023-02-21',
   'min_heart_rate': 71,
   'max_heart_rate': 136,
   'mean_heart_rate':

In [10]:
with open("data/heart.json", "w") as outfile:
    json.dump(heart, outfile)